In [1]:
import pandas as pd
data = pd.read_csv("training.1600000.processed.noemoticon.csv", encoding="latin-1") # The Sentiment140 dataset

In [2]:
from os import listdir

neutral_ = []

neutral = listdir("neutral") # the directory with neutral data
for file in neutral:
    with open("neutral/"+file, "r", encoding="utf-8") as f:
        for line in f:
            neutral_.append(line)

In [3]:
from nltk import TweetTokenizer
tknzr = TweetTokenizer(strip_handles=True, reduce_len=True)

def word_features(tweet):
    features = tknzr.tokenize(tweet)
    return dict([(word), True] for word in features)

tweets = data.iloc[:,5]

neg_tweets = [(word_features(tweet), "neg") for tweet in tweets[:10000]]
pos_tweets = [(word_features(tweet), "pos") for tweet in tweets[800000:810000]]
neu_tweets = [(word_features(tweet), "neu") for tweet in neutral_]
# taking only 20000 tweets, plus ~3500 neutral sentences

In [4]:
prep_set = neg_tweets + pos_tweets + neu_tweets
import random

random.seed(123456)
random.shuffle(prep_set)

train = prep_set[:20000]
test = prep_set[20000:]

In [5]:
from nltk.classify import NaiveBayesClassifier
import nltk.classify.util
classifier = NaiveBayesClassifier.train(train)

In [6]:
print ('accuracy:', nltk.classify.util.accuracy(classifier, test))
classifier.show_most_informative_features()
classifier.classify(word_features("""Text analysis runs simple procedures on the submitted (English) text and visualizes the features extracted from it.
"""))

accuracy: 0.751937984496124
Most Informative Features
                   Trump = True              neu : pos    =    333.4 : 1.0
                shutdown = True              neu : pos    =    154.6 : 1.0
                       ? = True              pos : neu    =    101.0 : 1.0
               President = True              neu : neg    =     97.7 : 1.0
                   Cohen = True              neu : pos    =     76.0 : 1.0
                    it's = True              neg : neu    =     70.0 : 1.0
                   State = True              neu : pos    =     67.0 : 1.0
               president = True              neu : pos    =     65.3 : 1.0
                  former = True              neu : neg    =     63.6 : 1.0
                   among = True              neu : pos    =     63.5 : 1.0


'neu'

In [7]:
import pickle
with open("NB.pickle", "wb") as f:
    pickle.dump(classifier, f)